In [1]:
import os
from  dotenv import load_dotenv
load_dotenv()

True

In [2]:
if os.environ['OPEN_API_SECRET_KEY']:
    print("OpenAI API key is set")
else:
    print("OpenAI API key is not set")

OpenAI API key is set


## We will use local llm model ollama instead of openai paid, so any of you can test locally without spending bucks

#### Let's implement a RAG system using local LLMs via Ollama and interacting with a SQL database using LangChain.

In [20]:
## incase you are using openai api
# from langchain_openai import ChatOpenAI 
# llm = ChatOpenAI(model_name="gpt-<version>", temperature=0)

# from langchain_ollama import ChatOllama
## incase you are using ollama api
from langchain_community.chat_models import ChatOllama
from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain


llm = ChatOllama(model="llama3", temperature=0)
database = SQLDatabase.from_uri("sqlite:///demo.db")
chain = SQLDatabaseChain(llm=llm, database=database, verbose=True, return_direct=True)

query = ("What is the total revenue of Headphones, aggregate on product name?")

result = chain.run(query)
print("Result: ", result)




> Entering new SQLDatabaseChain chain...
What is the total revenue of Headphones, aggregate on product name?
SQLQuery:Here's the answer:

Question: What is the total revenue of Headphones, aggregate on product_name?
SQLQuery:
SELECT "product_name", SUM("price") AS "Total Revenue"
FROM "Sales"
WHERE "product_name" = 'Headphones'
GROUP BY "product_name";
SQLResult: [('Headphones', 200.0)]
> Finished chain.
Result:  [('Headphones', 200.0)]


# Let's implement a RAG system using local LLMs via Ollama and interacting with a SQL database using LangChain.
#### Step 1: We will pass our text to the LLM and get the response. (Prepare Document for your text data)

In [29]:
from langchain_core.documents import Document

# Your text data
my_text = """YRetrieval-augmented generation (RAG) is a technique that enables large language models (LLMs) to 
retrieve and incorporate new information from external data sources.[1] With RAG, LLMs do not respond to user 
queries until they refer to a specified set of documents. These documents supplement information from the LLM's pre-existing t
raining data.[2] This allows LLMs to use domain-specific and/or updated information that is not available in the training data.[2] 
For example, this helps LLM-based chatbots access internal company data or generate responses based on authoritative sources."""

doc = Document(page_content=my_text, metadata={"source": "wikipedia", "version": 1})

print(doc)

page_content='YRetrieval-augmented generation (RAG) is a technique that enables large language models (LLMs) to 
retrieve and incorporate new information from external data sources.[1] With RAG, LLMs do not respond to user 
queries until they refer to a specified set of documents. These documents supplement information from the LLM's pre-existing t
raining data.[2] This allows LLMs to use domain-specific and/or updated information that is not available in the training data.[2] 
For example, this helps LLM-based chatbots access internal company data or generate responses based on authoritative sources.' metadata={'source': 'wikipedia', 'version': 1}


#### Step 2: Splitting the document into chunks

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

chunks = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=25,
    length_function=len,
    is_separator_regex=False,
).split_documents([doc])

chunks

[Document(metadata={'source': 'wikipedia', 'version': 1}, page_content="YRetrieval-augmented generation (RAG) is a technique that enables large language models (LLMs) to \nretrieve and incorporate new information from external data sources.[1] With RAG, LLMs do not respond to user \nqueries until they refer to a specified set of documents. These documents supplement information from the LLM's pre-existing t\nraining data.[2] This allows LLMs to use domain-specific and/or updated information that is not available in the training data.[2]"),
 Document(metadata={'source': 'wikipedia', 'version': 1}, page_content='For example, this helps LLM-based chatbots access internal company data or generate responses based on authoritative sources.')]